# Contest

ClassiFicon Hackathon 2021 by **I’mbesideyou Inc.**

Face Expression Detection

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import cv2
import imutils
from imutils import face_utils
import dlib
import os
from zipfile import ZipFile

# Importing Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras import layers

Connecting Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Contest

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Contest


### Importing Dataset

Unzip Images

```
with ZipFile('happy_images.zip', 'r') as zipObj:
   zipObj.extractall()
```



In [ ]:
with ZipFile('happy_images.zip', 'r') as zipObj:
   zipObj.extractall()

Extracting Labels

In [ ]:
# Train Labels
df_Train = pd.read_csv("train.csv", names=['Image', 'Label'])
Train_Images = df_Train['Image'].values
y_Train = df_Train['Label'].values

# Test Labels
df_Valid = pd.read_csv("test.csv", names=['Image', 'Label'])
Valid_Images = df_Valid['Image'].values
y_Valid = df_Valid['Label'].values

Map Labels

In [ ]:
def EncodeLabels(Labels):
    LabelMap = {"NOT smile":[1,0,0], "positive smile":[0,1,0], "negative smile":[0,0,1]}
    y = []

    for i in range(Labels.shape[0]):
        y.append(LabelMap[Labels[i]])

    return np.array(y)

y_Train = EncodeLabels(y_Train)
y_Valid = EncodeLabels(y_Valid)

Extracting Images

In [ ]:
def ExtractImages(ImageNames):
    X = []
    for i in range(ImageNames.shape[0]):
        img = pimg.imread("happy_images/" + ImageNames[i] + ".jpg")
        img = cv2.resize(img,(192,192))
        X.append(img)

    return np.array(X)

X_Train = ExtractImages(Train_Images)
X_Valid = ExtractImages(Valid_Images)

### Preprocessing Images

Display Images

In [ ]:
def DisplayImages(Data):
    N = Data.shape[0]
    i = np.random.randint(N)

    plt.figure(figsize=(6,8))
    plt.imshow(Data[i])
    plt.show()

Contrast Correction

In [ ]:
def ContrastCorrection(Dataset):
    Data = []
    for i in range(Dataset.shape[0]):
        r_image, g_image, b_image = cv2.split(Dataset[i])
        r_image_eq = cv2.equalizeHist(r_image)
        g_image_eq = cv2.equalizeHist(g_image)
        b_image_eq = cv2.equalizeHist(b_image)
        image_eq = cv2.merge((r_image_eq, g_image_eq, b_image_eq))
        Data.append(image_eq)

    return np.array(Data)

X_Train = ContrastCorrection(X_Train)
X_Valid = ContrastCorrection(X_Valid)

### Key Point Detection




In [ ]:
def LandMarks(image):
    Regions = []

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            clone = image.copy()
            cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 255), 2)
            for (x, y) in shape[i:j]:
                cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
            (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
            roi = image[y:y + h, x:x + w]
            roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            
            roi = cv2.resize(roi,(150,100))
            Regions.append(roi)

    return np.array(Regions)

### Region Extraction




In [ ]:
def Extraction(Images,Labels):
    X,y = [],[]
    for i in range(Images.shape[0]):
        print (i)
        try:
            Regions = LandMarks(Images[i])
            if Regions.shape != (8,100,150,3):
                img = cv2.resize(Images[i],(150,100))
                Regions = np.repeat(np.expand_dims(img,axis=0),8,axis=0)
            X.append(Regions)
            y.append(Labels[i])
        except:
            img = cv2.resize(Images[i],(150,100))
            Regions = np.repeat(np.expand_dims(img,axis=0),8,axis=0)
            X.append(Regions)
            y.append(Labels[i])

    return np.array(X),np.array(y)

In [ ]:
X_TrainFeatures,y_Train = Extraction(X_Train,y_Train)
X_ValidFeatures,y_Valid = Extraction(X_Valid,y_Valid)

np.save("XTrain.npy", X_Train)
np.save("XValid.npy", X_Valid)
np.save("XTrainFeatures.npy", X_TrainFeatures)
np.save("XValidFeatures.npy", X_ValidFeatures)

np.save("yTrain.npy", y_Train)
np.save("yValid.npy", y_Valid)